IMPORTS AND INSTALL

In [58]:
# imports
from PyPDF2 import PdfReader
import pandas as pd

# local do arquivo
file_path = 'RHPP05LA.PDF'

VARIÁVEIS

In [59]:
# correcao de dados
SUBSTITUICOES = {
    "1/12 FERIAS"           : "ferias", 
    "1/12 ferias"           : "ferias", 
    "1/12 DE 1 /3 FERIAS"   : "13_ferias", 
    "1 /3 ferias"           : "13_ferias", 
    "1 /3 FERIAS"           : "13_ferias", 
    "1/12 DE 13_ferias"     : "13_ferias", 
    "TOTAL ferias"          : "total_ferias", 
    "TOTAL FERIAS"          : "total_ferias", 
    "1/12 13 SAL ARIO"      : "13_salario", 
    "13 SAL ARIO"           : "13_salario", 
    "T O T A L"             : "total", 
    "PROVISIONAMENTO"       : "apropriacao", 
    "INSS  20,00%"          : "inss", 
    "SP-PREVCOM"            : "sp-prevcom", 
    "AC TRAB  "             : "ac_trab_", 
    "FGTS 8%"               : "fgts", 
    "TOTAL DE ENCARGOS"     : "total_encargos", 
    "."                     : "", 
    ","                     : ".", 
    "SPPREV"                : "spprev", 
    "FERIAS"                : "ferias"
}

FUNÇÕES

In [60]:
# funcao que divide uma lista em duas listas
# de acordo com a palavra enviada e os pads definidos
def split_tables(
        tb: list, 
        split_word:str="", 
        pad_top_tb1:int=0,
        pad_bottom_tb1:int=0,
        pad_top_tb2:int=0,
        pad_bottom_tb2:int=0
        ):
    
    for i in range(len(tb)):
        if split_word in tb[i]:
            tb1 = tb[pad_top_tb1:i+pad_bottom_tb1]
            tb2 = tb[i+pad_top_tb2:] if pad_bottom_tb2==0 else tb[i+pad_top_tb2:pad_bottom_tb2] 
    return tb1, tb2

# funcao que normatiza palavras-campos das tabelas
def correct_words(tb: list):
    for i in range(len(tb)):
        for search, correct in SUBSTITUICOES.items():
            tb[i] = tb[i].replace(search, correct.lower())
    return tb

# funcao que corrige os dados numericos quebrados por espaco
def data_processing(datas:list):
    for data in datas:
        i = 0
        while i < len(data) - 1:
            if "." not in data[i]:
                data[i] += data[i+1]
                del data[i+1]
            else:
                i += 1
    return datas

# funcao que extrai cabecalho, itens e dados da lista 
# formato tabela: 
## linha    0:  cabecalhos
## coluna   0:  itens
## coluna 1-5:  dados
def extract_tables(tb:list):
    tb = correct_words(tb)

    headers = tb[0].split()

    datas = tb[1:]
    for i in range(len(datas)):
        datas[i] = datas[i].split()
    
    items = [data[0] for data in datas]

    datas = [data[1:] for data in datas]
    datas = data_processing(datas)

    return headers, items, datas

def cast_table_to_df(tb:list) -> pd.DataFrame:
    headers, items, datas = extract_tables(tb)

    dictionary = {' ': items}

    for header, collumn in zip(headers, zip(*datas)):
        dictionary[header] = float(collumn) if isinstance(collumn, float) else collumn

    return pd.DataFrame(dictionary)



# funcao que coloca a tabela em formato de tupla
def cast_table_to_tuple(tb:list) -> tuple:
    headers, items, datas = extract_tables(tb)
    
    tup = {}
    
    for type, data in zip(items, datas):
        tup[type] = {}
        for header, value in zip(headers, data):
            tup[type][header] = float(value)

    return tup   

EXECUÇÃO PRINCIPAL

In [61]:
unidades = {}

# ler pdf
with open(file_path, 'rb') as pdf_file:
    pdf_reader = PdfReader(pdf_file)
    pages = pdf_reader.pages

    # para cada page indexada com um number_page
    # extrair texto e colocar em lines (retirando as em branco)
    for number_page, page in enumerate(pages, start=1):
        text = page.extract_text()
        lines = text.split("\n")
        lines = [line for line in lines if not line.isspace()]

        # extrai dados da unidade
        unidade = {}
        unidade['codigo'] = int(lines[3].strip().split(" - ")[0].replace(" ", ""))
        unidade['nome'] = lines[3].strip().split(" - ")[1]

        # se a pagina for impar, os dados sao referentes apropriado e realizado da unidade
        # se a pagina for par, os dados sao referentes aos totais da unidade
        if(number_page %2 == 1):

            # extrai tabela CLT e Autarquico
            tb_clt, tb_aut = split_tables(lines, "A U T A R Q U I C O", 7, -1, 2, -1)

            # extrai apropriado e realizado da tabela
            tb_clt_apropriado, tb_clt_realizado = split_tables(tb_clt, "R E V E R S A O", 1, 0, 1)
            tb_aut_apropriado, tb_aut_realizado = split_tables(tb_aut, "R E V E R S A O", 1, 0, 1)

            # gera dataframe para as tabelas
            tb_clt_apropriado   = cast_table_to_df(tb_clt_apropriado)
            tb_clt_realizado    = cast_table_to_df(tb_clt_realizado)
            tb_aut_apropriado   = cast_table_to_df(tb_aut_apropriado)
            tb_aut_realizado    = cast_table_to_df(tb_aut_realizado)

            display(str(unidade["codigo"])+" - "+unidade["nome"], "CLT", tb_clt_apropriado, tb_clt_realizado, "AUTÁRQUICO", tb_aut_apropriado, tb_aut_realizado)

'34240 - C.RIO CLARO-IB'

'CLT'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,121536.56,40512.20,162048.76,121536.56,283585.32
1,inss,20141.29,6713.74,26855.03,20141.29,46996.32
2,ac_trab_0.5000%,503.15,167.71,670.86,503.15,1174.01
3,fgts,8056.28,2685.46,10741.74,8056.28,18798.02
4,sp-prevcom,0.00,0.00,0.00,2244.59,2244.59
5,total_encargos,28700.72,9566.91,38267.63,30945.31,69212.94
6,total,150237.28,50079.11,200316.39,152481.87,352798.26


,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,196436.52,65477.99,261914.51,8254.74,270169.25
1,inss,36138.03,12045.20,48183.23,1650.78,49834.01
2,ac_trab_0.5000%,903.98,300.49,1204.47,41.04,1245.51
3,fgts,14497.57,4831.55,19329.12,660.09,19989.21
4,sp-prevcom,0.00,0.00,0.00,0.00,0.00
5,total_encargos,51539.58,17177.24,68716.82,2351.91,71068.73
6,total,247976.10,82655.23,330631.33,10606.65,341237.98


'AUTÁRQUICO'

...

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,189086.18,63028.79,252114.97,189086.18,441201.15
1,inss,28138.69,9379.50,37518.19,28138.69,65656.88
2,ac_trab_0.5000%,702.74,234.26,937.00,702.74,1639.74
3,fgts,11255.03,3751.66,15006.69,11255.03,26261.72
4,sp-prevcom,0.00,0.00,0.00,2598.55,2598.55
5,total_encargos,40096.46,13365.42,53461.88,42695.01,96156.89
6,total,229182.64,76394.21,305576.85,231781.19,537358.04


,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,355099.47,118365.12,473464.59,1398.34,474862.93
1,inss,64466.55,21487.46,85954.01,279.66,86233.67
2,ac_trab_0.5000%,1611.85,536.32,2148.17,6.98,2155.15
3,fgts,25835.30,8610.41,34445.71,111.86,34557.57
4,sp-prevcom,0.00,0.00,0.00,0.00,0.00
5,total_encargos,91913.70,30634.19,122547.89,398.50,122946.39
6,total,447013.17,148999.31,596012.48,1796.84,597809.32


'AUTÁRQUICO'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,424601.22,141533.72,566134.94,424601.22,990736.16
1,spprev,79748.71,0.00,79748.71,79748.71,159497.42
2,sp-prevcom,0.00,0.00,0.00,800.42,800.42
3,total_encargos,79748.71,0.00,79748.71,80549.13,160297.84
4,total,504349.93,141533.72,645883.65,505150.35,1151034.00


,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,0.00,0.00,0.00,0.00,0.00
1,spprev,0.00,0.00,0.00,0.00,0.00
2,sp-prevcom,0.00,0.00,0.00,0.00,0.00
3,total_encargos,0.00,0.00,0.00,0.00,0.00
4,total,0.00,0.00,0.00,0.00,0.00
